In [1]:
import asyncio  # Import asyncio for managing async execution
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import LiquidHandlerChatterboxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.resources.hamilton import STARLetDeck

from pylabrobot.resources.vwr import VWRReagentReservoirs25mL

# liquid handling
from pylabrobot.resources import set_tip_tracking, set_volume_tracking
set_tip_tracking(True), set_volume_tracking(True)

# setup
lh = LiquidHandler(backend=LiquidHandlerChatterboxBackend(), deck=STARLetDeck(with_teaching_rack=False))

await lh.setup()
vis = Visualizer(resource=lh)
await vis.setup()

# HAVE TO REFRESH THE PAGE FOR THE DECK TO APPEAR PROPERLY

Setting up the liquid handler.
Resource deck was assigned to the liquid handler.
Resource trash was assigned to the liquid handler.
Resource trash_core96 was assigned to the liquid handler.
Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


In [2]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cor_96_wellplate_360ul_Fb,
    Trough_CAR_4R200_A00,
    HTF
)

# Assign 5 tip racks to tip carrier
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = tip_rack1 = HTF(name='tips_01', with_tips=False)
tip_car[1] = tip_rack2 = HTF(name='tips_02', with_tips=False)
tip_car[2] = tip_rack3 = HTF(name='tips_03', with_tips=False)
tip_car[3] = tip_rack4 = HTF(name='tips_04', with_tips=False)
tip_car[4] = tip_rack5 = HTF(name='tips_05', with_tips=False)
# Assign tip carrier to lh (liquid handler)
lh.deck.assign_child_resource(tip_car, rails=15)

Resource tip carrier was assigned to the liquid handler.


In [3]:
# Assign 3 plates to plate carrier
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = plate_1 = Cor_96_wellplate_360ul_Fb(name='plate_01')
plt_car[1] = plate_2 = Cor_96_wellplate_360ul_Fb(name='plate_02')
plt_car[2] = plate_3 = Cor_96_wellplate_360ul_Fb(name='plate_03')
# Assign plate carrier to lh (liquid handler)
lh.deck.assign_child_resource(plt_car, rails=8)

Resource plate carrier was assigned to the liquid handler.


In [4]:
# Fill all tip_rack1 tips
tip_rack1.fill()
# Fill the top two rows of tip_rack4
tip_rack4 = lh.deck.get_resource("tips_04")
tip_rack4.set_tip_state(([True, True] + [False] * 6) * 12)
# fill alternating rows
flattened_bool_list = [True, False] * 48
tip_rack2.set_tip_state(flattened_bool_list)

In [5]:
# Setup trough carrier for material
trough_car = Trough_CAR_4R200_A00(name='trough carrier')
compounds = ['material_A', 'material_B', 'material_C', 'material_D']
compound_troughs = {}
for i, compound in enumerate(compounds):
  trough_car[i] = compound_troughs[compound] = VWRReagentReservoirs25mL(name=f'{compound}')
# Assign trough carrier to lh (liquid handler)
lh.deck.assign_child_resource(trough_car, replace=True, rails=4)

Resource trough carrier was assigned to the liquid handler.


In [6]:
from pylabrobot.resources.liquid import Liquid
import csv

def read_csv(file_path):
  with open(file_path, 'r') as file:
      reader = csv.DictReader(file)
      data = [row for row in reader]
  return data
CSV_FILE_PATH = '/Users/evan_kim/startup/pylabrobot/protocol-designer/backend/structured96.csv'
data = read_csv(CSV_FILE_PATH)

# Set liquids in reservoirs
def calculate_total_volume(data, compound):
    """Calculate total volume needed for a compound including extra buffer"""
    total = sum(float(row[compound]) for row in data if row[compound])
    # Add 10% buffer volume
    return total * 1.1

for compound in compounds:
  total_volume = calculate_total_volume(data, compound)
  print(total_volume)
  print("max", compound_troughs[compound].max_volume)
  if total_volume > compound_troughs[compound].max_volume:
      raise ValueError(f"Required volume {total_volume}μL for {compound} exceeds trough capacity {compound_troughs[compound].max_volume}μL")
  # logging.info(f"Filling trough for {compound} with {total_volume}μL")
  compound_troughs[compound].tracker.set_liquids([(compound, 20000)])

924.0000000000001
max 25000
1584.0000000000002
max 25000
1320.0
max 25000
1636.8000000000002
max 25000


In [8]:
# Set liquids in reservoirs
plate_1_liquids = [[("material_A", 500)]]*96
plate_1.set_well_liquids(plate_1_liquids)

plate_2_liquids = [[("material_B", 100)], [(None, 500)]]*(96//2)
plate_2.set_well_liquids(plate_2_liquids)

In [30]:

await lh.pick_up_tips(tip_rack1["A1", "B2", "C3", "D4"])

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tips_01_tipspot_1_1  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tips_01_tipspot_2_2  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tips_01_tipspot_3_3  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


In [31]:
await lh.drop_tips(tip_rack1["A1", "B2", "C3", "D4"])

Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tips_01_tipspot_1_1  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tips_01_tipspot_2_2  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tips_01_tipspot_3_3  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


In [32]:
# aspirate and dispensing
await lh.pick_up_tips(tip_rack1["A1"])

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


In [33]:
await lh.aspirate(plate_1["A2"], vols=[200])

Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 200.0    plate_01_well_1_0    0,0,0            None       None       None       


In [34]:
await lh.dispense(plate_2["A1"], vols=[200])

Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 200.0    plate_02_well_0_0    0,0,0            None       None       None       


In [35]:
await lh.drop_tips(tip_rack1["A1"])

Dropping tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


In [36]:
await lh.pick_up_tips(tip_rack1["A1", "B2", "C3", "D4"])

Picking up tips:
pip#  resource             offset           tip type     max volume (µL)  fitting depth (mm)   tip length (mm)  filter    
  p0: tips_01_tipspot_0_0  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p1: tips_01_tipspot_1_1  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p2: tips_01_tipspot_2_2  0,0,0            HamiltonTip  1065             8                    95.1             Yes       
  p3: tips_01_tipspot_3_3  0,0,0            HamiltonTip  1065             8                    95.1             Yes       


In [37]:
await lh.aspirate(plate_1["A1", "B2", "C3", "D4"], vols=[200, 100, 300, 50])

Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 200.0    plate_01_well_0_0    0,0,0            None       None       None       
  p1: 100.0    plate_01_well_1_1    0,0,0            None       None       None       
  p2: 300.0    plate_01_well_2_2    0,0,0            None       None       None       
  p3: 50.0     plate_01_well_3_3    0,0,0            None       None       None       


In [38]:
await lh.dispense(plate_3["A1", "A2", "A3", "A4"], vols=[200, 100, 300, 50])

Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 200.0    plate_03_well_0_0    0,0,0            None       None       None       
  p1: 100.0    plate_03_well_1_0    0,0,0            None       None       None       
  p2: 300.0    plate_03_well_2_0    0,0,0            None       None       None       
  p3: 50.0     plate_03_well_3_0    0,0,0            None       None       None       


In [39]:
[well for well in plate_3.children if well.name == "plate_03_well_3_0"][0]
d = plate_3.serialize_all_state()
d.keys()
for i in range(4):
    print(d[f"plate_03_well_{i}_0"])

{'liquids': [[None, 200.0]], 'pending_liquids': [[None, 200.0]], 'liquid_history': []}
{'liquids': [[None, 100.0]], 'pending_liquids': [[None, 100.0]], 'liquid_history': []}
{'liquids': [[None, 300.0]], 'pending_liquids': [[None, 300.0]], 'liquid_history': []}
{'liquids': [[None, 50.0]], 'pending_liquids': [[None, 50.0]], 'liquid_history': []}


In [40]:
# await lh.aspirate96(plate_1, volume=100)
# await lh.dispense96(plate_3, volume=100)
# await lh.drop_tips96(tip_rack1)
# await vis.stop()

d2 = plate_2.serialize_state()
d3 = plate_2.serialize()

print(d.keys())
print(d2.keys())
print(d3.keys())

print(d3["children"])
print(d["plate_03_well_0_1"].keys())

dict_keys(['plate_03', 'plate_03_well_0_0', 'plate_03_well_0_1', 'plate_03_well_0_2', 'plate_03_well_0_3', 'plate_03_well_0_4', 'plate_03_well_0_5', 'plate_03_well_0_6', 'plate_03_well_0_7', 'plate_03_well_1_0', 'plate_03_well_1_1', 'plate_03_well_1_2', 'plate_03_well_1_3', 'plate_03_well_1_4', 'plate_03_well_1_5', 'plate_03_well_1_6', 'plate_03_well_1_7', 'plate_03_well_2_0', 'plate_03_well_2_1', 'plate_03_well_2_2', 'plate_03_well_2_3', 'plate_03_well_2_4', 'plate_03_well_2_5', 'plate_03_well_2_6', 'plate_03_well_2_7', 'plate_03_well_3_0', 'plate_03_well_3_1', 'plate_03_well_3_2', 'plate_03_well_3_3', 'plate_03_well_3_4', 'plate_03_well_3_5', 'plate_03_well_3_6', 'plate_03_well_3_7', 'plate_03_well_4_0', 'plate_03_well_4_1', 'plate_03_well_4_2', 'plate_03_well_4_3', 'plate_03_well_4_4', 'plate_03_well_4_5', 'plate_03_well_4_6', 'plate_03_well_4_7', 'plate_03_well_5_0', 'plate_03_well_5_1', 'plate_03_well_5_2', 'plate_03_well_5_3', 'plate_03_well_5_4', 'plate_03_well_5_5', 'plate_03_w

In [41]:
await lh.aspirate(plate_3["A1", "A2", "A3", "A4"], vols=[50, 50, 50, 50])

Aspirating:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 50.0     plate_03_well_0_0    0,0,0            None       None       None       
  p1: 50.0     plate_03_well_1_0    0,0,0            None       None       None       
  p2: 50.0     plate_03_well_2_0    0,0,0            None       None       None       
  p3: 50.0     plate_03_well_3_0    0,0,0            None       None       None       


In [42]:
await lh.dispense(plate_3["C1", "C2", "C3", "C4"], vols=[50, 50, 50, 50])

Dispensing:
pip#  vol(ul)  resource             offset           flow rate  blowout    lld_z       
  p0: 50.0     plate_03_well_0_2    0,0,0            None       None       None       
  p1: 50.0     plate_03_well_1_2    0,0,0            None       None       None       
  p2: 50.0     plate_03_well_2_2    0,0,0            None       None       None       
  p3: 50.0     plate_03_well_3_2    0,0,0            None       None       None       


In [ ]:
await lh.aspirate(plate_3["A1", "A2", "A3", "A4"], vols=[50, 50, 50, 50])